Extract data from a DAO NetCDF file

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
ds = xr.open_dataset("dao.80_93.nc")

In [ ]:
if ds.coords["level"].isnull().all():
    print("Fixing broken pressure level data")
    levels = [1000.0, 950.0, 900.0, 850.0, 700.0, 500.0, 300.0, 200.0]
    ds.coords["level"] = xr.DataArray(levels, dims=["level"], coords={"level": levels}, attrs={"units": "hPa"})

In [ ]:
# make sure lat runs from south to north
if not ds["lat"].to_index().is_monotonic_increasing:
    print("flipping lat")
    ds = ds.sortby("lat", ascending=True)

In [ ]:
# make sure lon runs from west to east
if not ds["lon"].to_index().is_monotonic_increasing:
    print("flipping lon")
    ds = ds.sortby("lon", ascending=True)

In [ ]:
# get a spatial subset 4°N–16°S, 50°–76°W
ds = ds.sel(lat=slice(-16, 4), lon=slice(-76, -50))

In [ ]:
# make sure that the order of the dimensions is (lon, lat, ...) for all variables
ds = ds.transpose("lon", "lat", ...)

In [ ]:
# grab the first time step
# should be Jan 1980
ds = ds.isel(time=0, drop=True)

In [ ]:
import bulk_recycling_model.numerical_integration

In [ ]:
# Integrate 10^-3 Shum Uwnd dp
# Because the integration limits are from high pressure to low pressure, we need to invert the sign.
integrand = -1 * 1e-3 * ds["Shum"] * ds["Uwnd"]
Fx = bulk_recycling_model.numerical_integration.integrate_no_extrapolation(integrand, ds["Psfc"])
# Units: mb x m/s

In [ ]:
# Integrate 10^-3 Shum Vwnd dp
# Because the integration limits are from high pressure to low pressure, we need to invert the sign.
integrand = -1 * 1e-3 * ds["Shum"] * ds["Vwnd"]
Fy = bulk_recycling_model.numerical_integration.integrate_no_extrapolation(integrand, ds["Psfc"])
# Units: mb x m/s

Prepare and scale the data

In [ ]:
from bulk_recycling_model import preprocess_low_resolution
from bulk_recycling_model.axis import Axis
from bulk_recycling_model.scaling import Scaling, UnitSystem

In [ ]:
# degrees
L = ds.coords["lon"].max().item() - ds.coords["lon"].min().item()
# convert to meters
L = L * 111e3 * np.cos(np.deg2rad(ds.coords["lat"].mean().item()))
dx = L / ds.sizes["lon"]

In [ ]:
# degrees
H = ds.coords["lat"].values[-1] - ds.coords["lat"].values[0]
# convert to meters
H = H * 111e3
dy = H / ds.sizes["lat"]

In [ ]:
# adjust dx, dy for lower resolution secondary grid
dx = dx * 2
dy = dy * 2

In [ ]:
print(f"{L = :.2e} m")
print(f"{dx = :.2e} m")
print(f"{H = :.2e} m")
print(f"{dy = :.2e} m")

In [ ]:
# make a scaling object to convert between unit systems
scaling = Scaling(H)

In [ ]:
dx = scaling.distance.convert(dx, UnitSystem.SI, UnitSystem.scaled)
dy = scaling.distance.convert(dy, UnitSystem.SI, UnitSystem.scaled)
print(f"{dx = :.2e} scaled")
print(f"{dy = :.2e} scaled")

In [ ]:
# convert Fx and Fy to scaled units
Fx = scaling.water_vapor_flux.convert(Fx.values, UnitSystem.natural, UnitSystem.scaled)
Fy = scaling.water_vapor_flux.convert(Fy.values, UnitSystem.natural, UnitSystem.scaled)

In [ ]:
# preprocess water vapor fluxes onto the secondary grid
Fx_left = preprocess_low_resolution.prepare_Fx_left(Fx)
Fx_right = preprocess_low_resolution.prepare_Fx_right(Fx)
Fy_bottom = preprocess_low_resolution.prepare_Fy_bottom(Fy)
Fy_top = preprocess_low_resolution.prepare_Fy_top(Fy)

In [ ]:
# convert E to scaled units
E = scaling.evaporation.convert(ds["Evap"].values, UnitSystem.natural, UnitSystem.scaled)

In [ ]:
# preprocess E onto the secondary grid
E = preprocess_low_resolution.prepare_E(E)

In [ ]:
# compute P
P = preprocess_low_resolution.calculate_precipitation(Fx_left, Fx_right, Fy_bottom, Fy_top, E, dx, dy)

In [ ]:
N, M = E.shape
# lon axis
lon_axis = Axis(
    ds.coords["lon"].min().item(),
    ds.coords["lon"].diff("lon").mean().item() * 2,  # lower resolution step
    N + 1,
)
# lat axis
lat_axis = Axis(
    ds.coords["lat"].min().item(),
    ds.coords["lat"].diff("lat").mean().item() * 2,  # lower resolution step
    M + 1,
)

Run the model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("bulk_recycling_model").setLevel(logging.INFO)

In [ ]:
from bulk_recycling_model import plotting
from bulk_recycling_model.main import run

In [ ]:
class Callback:
    def __init__(self):
        self.n_calls = 0  # number of times callback has executed
        self.fig = plt.figure()  # reusable figure object

    def __call__(self, rho: np.ndarray, k: int):
        self.fig.clf()  # clear the figure for reuse
        ax = self.fig.add_subplot(1, 1, 1)
        collection = plotting.pcolormesh(ax, rho, lon_axis, lat_axis, vmin=0.0, vmax=0.8)
        self.fig.colorbar(collection)
        self.fig.suptitle(f"$\\rho^k$ @ k={k:04d}")
        plt.savefig(f"plots/{self.n_calls:05d}_{k:04d}.png")

        self.n_calls += 1

In [ ]:
! rm plots/*.png

In [ ]:
status = run(
    Fx_left,
    Fx_right,
    Fy_bottom,
    Fy_top,
    E,
    P,
    dx,
    dy,
    tol=1e-2,
    step_callback=Callback(),
)
assert status["success"]

Animate the plots using ffmpeg

```bash
ffmpeg -framerate 30 -pattern_type glob -i "*.png" -vf "tpad=stop_mode=clone:stop_duration=3" -c:v libx264 -pix_fmt yuv420p mov.mp4
```

In [ ]:
# plot the final solution
rho = status["rho"]
fig, ax = plt.subplots()
collection = plotting.pcolormesh(ax, rho, lon_axis, lat_axis, vmin=0.0, vmax=0.8)
fig.colorbar(collection)
fig.suptitle("$\\rho$")

In [ ]:
# plot the convergence
deltas = status["deltas"]
fig, ax = plt.subplots()
ax.plot(deltas)
ax.set_title("Convergence")
ax.set_xlabel("Iteration")